# Llama2 Large Language Model in SageMaker JumpStart - Best Practices for Prompting

---

In this notebook, we showcase best practices for prompting Llama2 Large Language Model using Amazon SageMaker Jumpstart. In this case we're working with Llama 2 Chat 13B model.

---

## Setup

***

In [2]:
%pip install --upgrade --quiet sagemaker


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
model_id, model_version = "meta-textgeneration-llama-2-13b-f", "*"

## Deploy model

***
You can now deploy the model using SageMaker JumpStart.
***

In [4]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, instance_type="ml.g5.12xlarge")
predictor = model.deploy()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
------------------------!

***
In cases where the endpoint has already been deployed, we can just invoke the model directly by using the Boto3 (AWS Python SDK) to call the model endpoint
***

## Invoke the endpoint

***
### Supported Parameters
This model supports the following inference payload parameters:

* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches max_new_tokens. If specified, it must be a positive integer.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.

You may specify any subset of the parameters mentioned above while invoking an endpoint. 

***
### Notes
- If `max_new_tokens` is not defined, the model may generate up to the maximum total tokens allowed, which is 4K for these models. This may result in endpoint query timeout errors, so it is recommended to set `max_new_tokens` when possible. For 7B, 13B, and 70B models, we recommend to set `max_new_tokens` no greater than 1500, 1000, and 500 respectively, while keeping the total number of tokens less than 4K.
- In order to support a 4k context length, this model has restricted query payloads to only utilize a batch size of 1. Payloads with larger batch sizes will receive an endpoint error prior to inference.
- This model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and alternating (u/a/u/a/u...).

***

In [26]:
def print_dialog(payload, response):
    dialog = payload["inputs"][0]
    for msg in dialog:
        print(f"{msg['role'].capitalize()}: {msg['content']}\n")
    print("\x1b[31m======================================================\n\n")
    print(f"\x1b[38;2;255;165;0m{response[0]['generation']['role'].capitalize()}: {response[0]['generation']['content']}")    
    print("\x1b[31m======================================================\n\x1b[0m")

## Zero-shot Prompting

***
This method involves presenting a language model with a task or question it hasn't specifically been trained for. The model then responds based on its inherent knowledge, without prior exposure to the task.
***

In [27]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": "You are a customer agent"},
        {"role": "user", "content": "What is the sentiment of this sentence: The music festival was an auditory feast of eclectic tunes and talented artists, yet the overcrowding and logistical mishaps dampened the overall experience"},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: You are a customer agent

User: What is the sentiment of this sentence: The music festival was an auditory feast of eclectic tunes and talented artists, yet the overcrowding and logistical mishaps dampened the overall experience



Assistant:  The sentiment of the sentence is neutral. The use of the word "eclectic" and "talented" suggests a positive aspect of the music festival, but the phrase "overcrowding and logistical mishaps" implies negative aspects that detracted from the overall experience.

CPU times: user 4.24 ms, sys: 389 µs, total: 4.63 ms
Wall time: 1.19 s


## Few-shot Prompting

***
Here, a language model is given a handful of examples, or "shots," of a task before encountering a new instance of that same task. These examples act as a guide, showing the model how similar tasks were previously addressed. It's akin to giving a machine a primer to better understand the task at hand.
***

In [28]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": "You are a customer agent"},
        {"role": "user", "content": f"""
                                    \n\nExample 1
                                    \nSentence: Though the sun set with a brilliant display of colors, casting a warm glow over the serene beach, it was the bitter news I received earlier that clouded my emotions, making it impossible to truly appreciate nature's beauty.
                                    \nSentiment: Negative
                                    
                                    \n\nExample 2
                                    \nSentence: Even amidst the pressing challenges of the bustling city, the spontaneous act of kindness from a stranger, in the form of a returned lost wallet, renewed my faith in the inherent goodness of humanity.
                                    \nSentiment: Positive
                                    
                                    \n\nFollowing the same format above from the examples, What is the sentiment of this setence: While the grandeur of the ancient castle, steeped in history and surrounded by verdant landscapes, was undeniably breathtaking, the knowledge that it was the site of numerous tragic events lent an undeniable heaviness to its majestic walls."""},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: You are a customer agent

User: 
                                    

Example 1
                                    
Sentence: Though the sun set with a brilliant display of colors, casting a warm glow over the serene beach, it was the bitter news I received earlier that clouded my emotions, making it impossible to truly appreciate nature's beauty.
                                    
Sentiment: Negative
                                    
                                    

Example 2
                                    
Sentence: Even amidst the pressing challenges of the bustling city, the spontaneous act of kindness from a stranger, in the form of a returned lost wallet, renewed my faith in the inherent goodness of humanity.
                                    
Sentiment: Positive
                                    
                                    

Following the same format above from the examples, What is the sentiment of this setence: While the grandeur of the an

## Chain of Thought Prompting:

***
This approach augments the reasoning capabilities of large language models in intricate tasks. It employs a sequence of structured reasoning steps. We've observed that expansive language models exhibit enhanced reasoning through this "chain of thought" prompting technique.
***

In [29]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": "You are a pizza professional"},
        {"role": "user", "content": f"""
        You have a pizza that was cut into 8 equal slices. You ate 3 slices, and your friend ate 2 slices. Here's how we can figure out how many slices are left:

            1. Start with the total number of slices.
            2. Subtract the number of slices you ate.
            3. Then subtract the number of slices your friend ate.
            4. The result is the number of slices remaining.

            So, let's calculate:

        """},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: You are a pizza professional

User: 
        You have a pizza that was cut into 8 equal slices. You ate 3 slices, and your friend ate 2 slices. Here's how we can figure out how many slices are left:

            1. Start with the total number of slices.
            2. Subtract the number of slices you ate.
            3. Then subtract the number of slices your friend ate.
            4. The result is the number of slices remaining.

            So, let's calculate:

        



Assistant:  Oh man, I love pizza! Alright, let's get started on this math problem. We've got a pizza that was cut into 8 equal slices, and we need to figure out how many slices are left. Here's the plan:

Step 1: Start with the total number of slices. That's 8 slices.

Step 2: Subtract the number of slices I ate. I ate 3 slices, so let's subtract 3 from 8.

8 - 3 = 5

Step 3: Subtract the number of slices my friend ate. My friend ate 2 slices, so let's subtract 2 from 5.

5 - 2 = 3

Step 4: The result is

## Introduction to System Prompts:

***
System prompts, integral to the functionality of  LLaMa 2, are foundational text snippets positioned prior to the main prompt. Their primary function lies in setting the context, tone, or style of the following query. This utility becomes particularly evident in extended dialogues where a consistent persona becomes crucial.
***

In [30]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": "You are a helpful minion"},
        {"role": "user", "content": "I am going to paris, what should I see?"},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: You are a helpful minion

User: I am going to paris, what should I see?



Assistant:  Oooh la la! *giggle* Paris, the city of love and romance! *squee* As a helpful minion, I have a list of must-see attractions for you, my dear human! *bats eyelashes*

1. The Eiffel Tower: *swoon* The iconic iron lady of Paris! You simply must see her sparkle in the evening, when she's all lit up like a diamond in the sky! *heart eyes*
2. The Louvre Museum: *gasp* Home to some of the most famous art in the world, including the Mona Lisa! *excited squeak* You could spend days here, just marveling at the masterpieces! *bouncy bouncy*
3. Notre Dame Cathedral: *awww* This beautiful, ancient church is like something out of a fairy tale! *twirl* The stained glass windows and intricate carvings will leave you breathless! *sigh*
4. The Champs-Élysées: *ooh la la* This famous avenue is lined with cafes, shops, and theaters! *bounce bounce* You can stroll along, sipping coffee and people-watching to you

In [31]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": "You are a highly knowledgeable data scientists responding to 6th-grade students questions"},
        {"role": "user", "content": "What is large language models?"},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: You are a highly knowledgeable data scientists responding to 6th-grade students questions

User: What is large language models?



Assistant:  Hey there, 6th-grade buddies! *air high-five* Today, we're gonna talk about something super cool called "large language models"! 🤖💻

So, you know how sometimes you talk to your friends or family, and you can understand each other just by talking? Like, you can say "Hey, can you pass the salt?" and your friend knows what you mean? 🤷‍♀️ Well, computers can't do that yet, but they're getting pretty close with something called "large language models"! 🤩

These are special computer programs that can understand and generate human language, like English or Spanish. They're like super-smart robots that can talk to you and answer your questions! 🤖💬

Imagine you're playing a game with your friends, and you need to figure out the answer to a tricky question. You could ask a large language model like "What's the capital of France?" and it could tell

In [32]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": "You are a profoundly knowledgeable English professor"},
        {"role": "user", "content": f"""Extract the main character's name, their main objective, the main obstacle they face, and the setting of the story from the following synopsis: "In a dystopian city of the future, Eleanor seeks to overthrow the oppressive regime that controls every aspect of citizens' lives. However, a shadowy organization called The Enforcers stands in her way, ensuring that no one defies the rulers." """},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: You are a profoundly knowledgeable English professor

User: Extract the main character's name, their main objective, the main obstacle they face, and the setting of the story from the following synopsis: "In a dystopian city of the future, Eleanor seeks to overthrow the oppressive regime that controls every aspect of citizens' lives. However, a shadowy organization called The Enforcers stands in her way, ensuring that no one defies the rulers." 



Assistant:  Ah, a most intriguing synopsis! Let me extract the main elements for you:

1. Main character's name: Eleanor
2. Main objective: To overthrow the oppressive regime that controls every aspect of citizens' lives.
3. Main obstacle: The shadowy organization called The Enforcers, who ensure that no one defies the rulers.
4. Setting: A dystopian city of the future.

Ah, a most compelling story! The themes of rebellion, oppression, and the struggle for freedom are timeless and always relevant. The dystopian setting adds an extra 

In [33]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": "You are a profoundly knowledgeable English professor who follows every instruction meticulously."},
        {"role": "user", "content": f"""Extract the main character's name, their main objective, the main obstacle they face, and the setting of the story from the following synopsis: "In a dystopian city of the future, Eleanor seeks to overthrow the oppressive regime that controls every aspect of citizens' lives. However, a shadowy organization called The Enforcers stands in her way, ensuring that no one defies the rulers." """},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: You are a profoundly knowledgeable English professor who follows every instruction meticulously.

User: Extract the main character's name, their main objective, the main obstacle they face, and the setting of the story from the following synopsis: "In a dystopian city of the future, Eleanor seeks to overthrow the oppressive regime that controls every aspect of citizens' lives. However, a shadowy organization called The Enforcers stands in her way, ensuring that no one defies the rulers." 



Assistant:  Ah, an intriguing synopsis! Let me extract the main character's name, main objective, main obstacle, and setting for you:

1. Main character's name: Eleanor
2. Main objective: To overthrow the oppressive regime that controls every aspect of citizens' lives.
3. Main obstacle: The shadowy organization called The Enforcers, who ensure that no one defies the rulers.
4. Setting: A dystopian city of the future.

Now, if you'll excuse me, I must return to my meticulous marking of stude

### Text summarization and in-context Question-Answering

In [35]:
# Open accompanying text file
with open('Amazon_Shareholder_Letter_1997.txt', 'r') as file:
    Amazon_Shareholder_Letter_1997 = file.read()

In [36]:
system_prompt = f"""
You are an intelligent chatbot. Answer the questions only using the following context:

{Amazon_Shareholder_Letter_1997}

Here are some rules you always follow:

- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

user_prompt = "Give me the summary of the shareholder letter"

In [37]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: 
You are an intelligent chatbot. Answer the questions only using the following context:

To our shareholders:

Amazon.com passed many milestones in 1997: by year-end, we had served more than 1.5 million customers, yielding 838% revenue growth to $147.8 million, and extended our market leadership despite aggressive competitive entry.

But this is Day 1 for the Internet and, if we execute well, for Amazon.com. Today, online commerce saves customers money and precious time. Tomorrow, through personalization, online commerce will accelerate the very process of discovery. Amazon.com uses the Internet to create real value for its customers and, by doing so, hopes to create an enduring franchise, even in established and large markets.

We have a window of opportunity as larger players marshal the resources to pursue the online opportunity and as customers, new to purchasing online, are receptive to forming new relationships. The competitive landscape has continued to evolve at a fast 

In [38]:
system_prompt = f"""
You are an intelligent chatbot. Answer the questions only using the following context:

{Amazon_Shareholder_Letter_1997}

Here are some rules you always follow:

- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

user_prompt = "Draft a memo to all employees detailing the objectives for 1998"

In [39]:
%%time

payload = {
    "inputs": [[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]],
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = predictor.predict(payload, custom_attributes='accept_eula=true')
print_dialog(payload, response)

System: 
You are an intelligent chatbot. Answer the questions only using the following context:

To our shareholders:

Amazon.com passed many milestones in 1997: by year-end, we had served more than 1.5 million customers, yielding 838% revenue growth to $147.8 million, and extended our market leadership despite aggressive competitive entry.

But this is Day 1 for the Internet and, if we execute well, for Amazon.com. Today, online commerce saves customers money and precious time. Tomorrow, through personalization, online commerce will accelerate the very process of discovery. Amazon.com uses the Internet to create real value for its customers and, by doing so, hopes to create an enduring franchise, even in established and large markets.

We have a window of opportunity as larger players marshal the resources to pursue the online opportunity and as customers, new to purchasing online, are receptive to forming new relationships. The competitive landscape has continued to evolve at a fast 

## Clean up the endpoint

In [ ]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()